In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-04 13:01:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.16MB/s    in 0.2s    

2023-02-04 13:01:52 (6.16 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...|2015-08-31 00:00:00|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...| 

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

6908554
6908145
6908145


In [ ]:
# print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create the customers_table DataFrame
from pyspark.sql.functions import count
customers_df = df.groupby("customer_id").agg(count("customer_id"))

customers_df.show(20)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   18575725|                 4|
|   49109697|                24|
|   50062110|                 2|
|   37181454|                 6|
|   14075594|                 1|
|   18936548|                 3|
|    2376109|                 4|
|   33658169|                 2|
|   20616552|                 2|
|   36749966|                 1|
|   30528452|                 1|
|    8972743|                 1|
|   21900520|                 1|
|   47532241|                 1|
|   49974112|                 1|
|   16553261|                 5|
|   23970028|                 1|
|   17071380|                 1|
|   51873555|                 3|
|   42890397|                 1|
+-----------+------------------+
only showing top 20 rows



In [ ]:
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")

customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   18575725|             4|
|   49109697|            24|
|   50062110|             2|
|   37181454|             6|
|   14075594|             1|
|   18936548|             3|
|    2376109|             4|
|   33658169|             2|
|   20616552|             2|
|   36749966|             1|
|   30528452|             1|
|    8972743|             1|
|   21900520|             1|
|   47532241|             1|
|   49974112|             1|
|   16553261|             5|
|   23970028|             1|
|   17071380|             1|
|   51873555|             3|
|   42890397|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()

products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0016P392O|iMicro SP-IM942 M...|
|B00PPRYEZA|CoulVue Holder an...|
|B00VZXRRGG|Hexagon Network -...|
|B007GF9N96|Computer USB-Powe...|
|B00LCK3TEG|           Fire HD 6|
|B00KMOG5FS|Fintie 10.1" Tabl...|
|B00CFL5CRI|Cable Matters 2-P...|
|B008NP2AUC|USB USB-2, USB-3 ...|
|B00OAJ5N6I|Samsung 850 EVO -...|
|B00H02XN2I|Techno Earth Rubb...|
|B00YSC6UW8|5 Mm Thick Office...|
|B00CRXZPC6|Lenovo Ideatab S6...|
|B009SKTZFY|Bosign Tabletpill...|
|B003NZSX7G|Netgear N300 WiFi...|
|B0089JIDLC|Kingston ValueRAM...|
|B001E45XT4|Plantronics GameC...|
|B002IY4V9C|Swiss Gear Men's ...|
|B000UVRU6G|SteelSeries QcK G...|
|B00X5X3LQY|EVGA Pro SLI Brid...|
|B00L49X8E6|ASUS X551 15.6-in...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id","product_id","product_parent",to_date("review_date","yyyy-MM-dd").alias("review_date")])

review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100CF6VJEQNV8|   12711954|B00B5VY2WA|     350531264| 2013-06-11|
|R100CY2T8EP8SE|   50207409|B002XISTXM|     701633860| 2012-12-27|
|R100FWGX3289MO|   16085175|B001JSSDGU|      94821553| 2010-03-11|
|R100MG39QC10ZE|   51764545|B000RPEK7W|      49925319| 2009-01-02|
|R100SS4LU2A9MU|   43895145|B0055PTUE2|      15445263| 2013-04-19|
|R100UWA60RMCZO|    7512500|B00FRHTTIU|      16015786| 2015-08-26|
|R100V41SVB6WY6|    4243874|B00FGPANME|     693661460| 2014-06-03|
| R100XGEPEAKWE|   10533265|B004N3BH0C|     858109028| 2012-08-21|
|R1010CMGRTQT3T|   50515506|B00JG8GOWU|     816234934| 2015-01-04|
|R1019B2X98AOFD|   33693720|B007LBOCVY|     881978740| 2013-01-29|
|R101BJBEZVC78U|   43130690|B00000JDF8|     897742589| 2015-04-25|
|R101E1XWPS6GHI|   29023581|B003ELVLKU|     690846015| 2013-06

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100CF6VJEQNV8|          1|            4|          4|   N|                N|
|R100CY2T8EP8SE|          2|            3|          5|   N|                N|
|R100FWGX3289MO|          4|            0|          0|   N|                Y|
|R100MG39QC10ZE|          5|            2|          2|   N|                Y|
|R100SS4LU2A9MU|          4|            0|          0|   N|                Y|
|R100UWA60RMCZO|          5|            0|          0|   N|                Y|
|R100V41SVB6WY6|          1|            0|          0|   N|                Y|
| R100XGEPEAKWE|          5|            0|          0|   N|                Y|
|R1010CMGRTQT3T|          4|            0|          0|   N|                Y|
|R1019B2X98AOFD|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
# Store environmental variable
from getpass import getpass
password = getpass()

mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.c8qy85pnur1r.us-east-1.rds.amazonaws.com:5432/pgModule17"
config = {"user":"pgModule17", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)